In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from matplotlib import pyplot as plt
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10

# convert class vectors to binary class matrices
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


In [4]:
# Séquentiel

model = tf.keras.Sequential()
model.add(Dense(120, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train_one_hot,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test_one_hot))
score = model.evaluate(x_test, y_test_one_hot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 120)               94200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 80)                9680      
_________________________________________________________________
dropout_3 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                810       
Total params: 104,690
Trainable params: 104,690
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 57u

In [5]:
# Model subclassing

class MLP(tf.keras.Model):
    
    def __init__(self):
        super(MLP,self).__init__()
        self.dense1 = Dense(120,activation='relu',input_shape=(784,))
        self.dropout1 = Dropout(0.2)
        self.dense2 = Dense(80, activation='relu')
        self.dropout2 = Dropout(0.2)
        self.dense3 = Dense(10, activation='softmax')
        
    def call(self,x):
        x = self.dense1(x)
        x = self.dropout1(x)
        x = self.dense2(x)
        x = self.dropout2(x)
        return self.dense3(x)
        
    def compute_output_shape(self, input_shape):
        # You need to override this function if you want to use the subclassed model
        # as part of a functional-style model.
        # Otherwise, this method is optional.
        shape = tf.TensorShape(input_shape).as_list()
        shape = [shape[0], self.num_classes]
        print(shape)
        return tf.TensorShape(shape)

In [11]:
my_mlp2 = MLP()
my_mlp2.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
history = my_mlp2.fit(x_train,y_train_one_hot,batch_size=32,epochs=10, validation_data=(x_test, y_test_one_hot))
score = my_mlp2.evaluate(x_test,y_test_one_hot,verbose=0)
print("Test loss: {}".format(score[0]))
print("Test accuracy: {}".format(score[1]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.0350 - accuracy: 0.9891 - val_loss: 0.0846 - val_accuracy: 0.9810
Epoch 2/10
60000/60000 [==============================] - 3s 50us/sample - loss: 0.0307 - accuracy: 0.9898 - val_loss: 0.0912 - val_accuracy: 0.9808
Epoch 3/10
60000/60000 [==============================] - 3s 51us/sample - loss: 0.0341 - accuracy: 0.9890 - val_loss: 0.0875 - val_accuracy: 0.9817
Epoch 4/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0355 - accuracy: 0.9887 - val_loss: 0.0824 - val_accuracy: 0.9814
Epoch 5/10
60000/60000 [==============================] - 3s 51us/sample - loss: 0.0327 - accuracy: 0.9897 - val_loss: 0.0890 - val_accuracy: 0.9816
Epoch 6/10
60000/60000 [==============================] - 3s 52us/sample - loss: 0.0348 - accuracy: 0.9889 - val_loss: 0.0810 - val_accuracy: 0.9826
Epoch 7/10
60000/60000 [==============================] 

In [25]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
sgd = tf.keras.optimizers.SGD()
my_mlp = MLP()

In [26]:
def mlp_class(predictions):
    return np.argmax(predictions,axis = 1)

def score(y_true,predictions):
    predictions = mlp_class(predictions)
    return np.sum(y_true == predictions)/10000

In [27]:
# Gradient Tape
@tf.function
def train(data,labels):
    with tf.GradientTape() as tape:
        predictions = my_mlp(data)
        loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss, my_mlp.trainable_variables)
    sgd.apply_gradients(zip(gradients,my_mlp.trainable_variables))

In [32]:
EPOCHS = 10
BATCH_SIZE = 32

iterations = 60000//BATCH_SIZE

for epoch in range(EPOCHS):
    for iteration in range(iterations):
        train(x_train[BATCH_SIZE*(iteration-1):BATCH_SIZE*(iteration),:],
              y_train_one_hot[BATCH_SIZE*(iteration-1):BATCH_SIZE*(iteration),:])

In [33]:
predictions = my_mlp(x_test)
error = loss_object(y_test_one_hot,predictions)
accuracy = score(y_test,predictions)
print("Test loss: {}".format(error))
print("Accuracy: {}".format(accuracy))

Test loss: 0.08430710434913635
Accuracy: 0.9757
